# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import copy
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from unityagents import UnityEnvironment
from collections import namedtuple, deque

In [2]:
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe")

# get the default brain
wimbledon = env.brain_names[0]
brain = env.brains[wimbledon]

env_info = env.reset(train_mode=True)[wimbledon]

action_size = brain.vector_action_space_size
states = env_info.vector_observations
state_size = states.shape[1]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
ep_len = np.zeros((5), dtype='int32')
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[wimbledon]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        ep_len[i-1]+=1
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[wimbledon]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

## Hyperparameters

In [3]:
BUFFER_SIZE = int(1e5)     # replay buffer size
BATCH_SIZE = 256          # minibatch size
GAMMA = 0.99               # discount factor
TAU = 1e-3                 # for soft update of target parameters
LR_ACTOR = 1e-4            # learning rate of the actor 
LR_CRITIC = 3e-4           # learning rate of the critic
update_every = 150          # number of timesteps after which to run an update
SN = 0.5                  # starting value for additive noise scale (exploratory actions)
ND = 0.999               # noise decay rate (exploratory actions)
NM = 0.01                  # noise minimum to be maintained (exploratory actions)

## DDPG agents
____
### Define Actor and Critic

In [4]:
class Actor(nn.Module):
    def __init__(self, state_size, action_size, hu=(256, 128), activ_in = F.relu, activ_out = torch.tanh):
        super(Actor, self).__init__()
        
        self.activ_in = activ_in
        self.activ_out = activ_out
        
        self.input_layer = nn.Linear(state_size, hu[0])
        self.hl1 = nn.Linear(hu[0], hu[1])
        self.output_layer = nn.Linear(hu[-1], action_size)
        
    def forward(self, state):
        x = state
        x = self.activ_in(self.input_layer(x))
        x = self.activ_in(self.hl1(x))
        return self.activ_out(self.output_layer(x))  
    
class Critic(nn.Module):
    def __init__(self, state_size, action_size, hu=(256, 128), activ_in = F.relu):
        super(Critic, self).__init__()
        
        self.activ_in = activ_in
        
        self.input_layer = nn.Linear(state_size, hu[0])
        self.hl1 = nn.Linear(hu[0]+action_size, hu[1])
        self.output_layer = nn.Linear(hu[-1], 1)
        
    def forward(self, state, action):
        x = state
        u = action
        
        x = self.activ_in(self.input_layer(x))
        x = torch.cat((x, u), dim=1)
        x = self.activ_in(self.hl1(x))
        return self.output_layer(x)

### Define DDPG agent

In [ ]:
class DDPG():
    def __init__(self, state_size, action_size, num_agents=2):
        self.state_size = state_size
        self.action_size = action_size
        
        ### Initialise actor online and target networks
        self.actor_online = Actor(state_size, action_size).to(device)
        self.actor_target = Actor(state_size, action_size).to(device)
        self.actor_optimizer = optim.Adam(self.actor_online.parameters(), lr=LR_ACTOR)
        
        ### Initialise critic online and target networks
        self.critic_online = Critic(state_size, action_size).to(device)
        self.critic_target = Critic(state_size, action_size).to(device)
        self.critic_optimizer = optim.Adam(self.critic_online.parameters(), lr=LR_CRITIC)
        
        ### Ensure actor and online networks start off with same parameters
        self.equalize_OnlineTarget(self.actor_target, self.actor_online)
        self.equalize_OnlineTarget(self.critic_target, self.critic_online)
        
        ### Noise parameters for exploration
        self.noise_scale = start_noise
        self.noise_decay = noise_decay
        self.noise_min = noise_min
        
        self.add_noise = add_noise
        
        ### Replay buffer
        #self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE)
        
        ### Keep track of timesteps since last training update
        #self.update = 0
        
    def equalize_OnlineTarget(self, target, online):
        for target_param, online_param in zip(target.parameters(), online.parameters()):
            target_param.data.copy_(online_param.data)
            
    def generate_noise(self):
        noise = np.random.normal(loc=0, scale=self.noise_scale, size=self.action_size)
        self.noise_scale = max(self.noise_decay*self.noise_scale, self.noise_min)
        return noise